# Random Forests

### Introduction
As we've mentioned in the previous chapter (see [decision trees](decisionTrees.ipynb)), decision trees are often used, but not as the single, final, best method. You are far more likely to encounter the concept of decision trees in the context of **random forests**. Moreover, other techniques you *will* see being used together with DT and RF are *boosting* and *bagging*.  
In a more general sense, these are all **ensemble methods**. Ensemble methods are methods that combine the predictions of several base estimators (say, some decision trees) in order to improve generalizability / robustness over a single estimator. We will talk about these methods in more detail in the next chapter. Random Forests themselves are a type of ensemble method, and we will talk about them in this chapter. We introduce ensemble methods through RF, then we will talk about them in more detail in the next chapter.

## What is a random forest?

A random forest is a *collection of decision trees*, trained on different subsets of the data. The idea is that each tree in the forest is trained on a different subset of the data, and the predictions of each tree are combined to get the final prediction. The final prediction is the class that is most common among the predictions of the trees in the forest. The idea is to "*listen*" to the crowd, and take the majority vote.  

As such, all we need to do is have 3 components at the ready:
- A way to train a decision tree
- A way to sample the data (to get different subsets of the data)
- A way to combine the predictions of the trees

In [23]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
# Import sklearn iris
from sklearn.datasets import load_iris

In [24]:
# Code from previous chapter
# Gini impurity - it doesn't have to be complex because we only have 2 features; we really need to calculate only 1 probability, since the other one is 1 - p
def gini(p):
    #return 1 - p**2 - (1 - p)**2 
    # do the math and the equation is equal to 2 * p * (1 - p)
    return 2 * p * (1 - p)

def gini_impurity(y):
    p = np.sum(y == 0) / len(y)
    return gini(p)

# Recursive binary splitting
def split(X, y, d, value):
    # Split the data on feature 'd' at 'value'
    left_indices = (X[:, d] <= value)
    X_left, y_left = X[left_indices], y[left_indices]
    X_right, y_right = X[~left_indices], y[~left_indices]
    return X_left, X_right, y_left, y_right

# Find the best split based on the mean value of each feature
def find_best_split(X, y):
    best_gini = 1.0
    best_d, best_value = None, None
    for d in range(X.shape[1]): # for each feature
        value = np.mean(X[:, d]) # find the mean value
        X_left, X_right, y_left, y_right = split(X, y, d, value) # split the data based on the mean of that feature
        gini_left, gini_right = gini_impurity(y_left), gini_impurity(y_right) # calculate the gini impurity of the left and right splits
        gini = (len(y_left) / len(y)) * gini_left + (len(y_right) / len(y)) * gini_right # calculate the weighted gini impurity
        if gini < best_gini: # if the weighted gini impurity is less than the best gini impurity, update the best gini impurity (we have only 2 features, we could've calculated them directly but this is more general)
            best_gini = gini
            best_d = d # best feature index
            best_value = value # best value for that feature (the mean in our case)
    return best_d, best_value

# Decision tree class
class DecisionTree:
    def __init__(self, max_depth=2, majThreshold = 0.8):
        self.max_depth = max_depth
        self.root = None
        self.majThreshold = majThreshold
    def fit(self, X, y):
        self.root = self._grow_tree(X, y)
    def _grow_tree(self, X, y, depth=0):
        num_samples_per_class = [np.sum(y == i) for i in range(y.max() + 1)]
        most_samples_per_class = np.max(num_samples_per_class)
        predicted_class = np.argmax(num_samples_per_class)
        node = Node(predicted_class=predicted_class)
        if depth < self.max_depth and most_samples_per_class / np.sum(num_samples_per_class) < self.majThreshold:
            d, value = find_best_split(X, y)
            if d is not None:
                X_left, X_right, y_left, y_right = split(X, y, d, value)
                node.d = d
                node.value = value
                node.left = self._grow_tree(X_left, y_left, depth + 1)
                node.right = self._grow_tree(X_right, y_right, depth + 1)
        return node
    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])
    def _traverse_tree(self, x, node):
        if node.left is None and node.right is None:
            return node.predicted_class
        if x[node.d] <= node.value:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)
    def accuracy(self, X, y):
        return np.sum(self.predict(X) == y) / len(y)

# Node class
class Node:
    def __init__(self, predicted_class):
        self.predicted_class = predicted_class
        self.d = None
        self.value = None
        self.left = None
        self.right = None

In [25]:
# And use some new functions to create a random forest

def trainDecisionTree(X, y, maxDepth=1, majThreshold = 0.8):
    # Create a new decision tree
    tree = DecisionTree(maxDepth, majThreshold)
    # Train it
    tree.fit(X, y)
    # Return it
    return tree

def trainRandomForest(X, y, nTrees=10, maxDepth=1, majThreshold=0.7):
    # Create a list of trees
    trees = []
    # For each tree
    for i in range(nTrees):
            # Train a decision tree
            tree = trainDecisionTree(X[i], y[i], maxDepth, majThreshold)
            # Add it to the list of trees
            trees.append(tree)
    # Return the list of trees
    return trees

def predictRandomForest(trees, X):
    # Create a list of predictions
    predictions = []
    # For each tree
    for tree in trees:
            # Make a prediction
            prediction = tree.predict(X)
            # Add it to the list of predictions
            predictions.append(prediction)
    # Convert the list of predictions to a numpy array
    predictions = np.array(predictions)
    # Return the most common prediction for each sublist
    return np.array([np.bincount(prediction).argmax() for prediction in predictions.T])

In [26]:
# Function to sample our dataset, based on number of trees we want to create
def sampleDataset(X, y, nSamples):
    # Create a list of random indices
    indices = np.random.choice(len(X), int(nSamples))
    # Return the X and y samples
    return X[indices], y[indices]
# Create n random samples
def createRandomSamples(X, y, noTrees, sampleSize):
    # Create a list of samples
    samplesX = []
    samplesY = []
    # For each sample
    for i in range(noTrees):
            # Create a random sample
            sampleX, sampleY = sampleDataset(X, y, sampleSize)
            # Add it to the list of samples
            samplesX.append(sampleX)
            samplesY.append(sampleY)
    # Return the list of samples
    return samplesX, samplesY

In [27]:
# Get Iris dataset
X, y = load_iris(return_X_y=True)
# Sample our data to prepare for tree training
noTrees = 10
sampleSize = 50
Xsampled, ySampled = createRandomSamples(X, y, noTrees, sampleSize)

Note that we cannot control the seed of the random sequence generator we used above, so results will vary.

In [28]:
# Train noTrees trees as part of the random forest
maxDepth = 2
majThreshold = 0.8
trees = trainRandomForest(Xsampled, ySampled, noTrees, maxDepth, majThreshold)

In [29]:
# Let's see predictions now
predictions = predictRandomForest(trees, X)
# And the accuracy
print("Accuracy: ", np.sum(predictions == y) / len(y))

Accuracy:  0.8133333333333334


On my runs, I see around 0.74 accuracy; not that big of an improvement.  

This is with majThreshold = 0.8 (80% of the trees need to vote for the same class) & maxDepth = 2 & 10 trees & 50 sample size.

Running with other values for the parameters might improve score, but not by much compared to the single decision tree with a bigger depth. So we need to improve in other regards.


### Sampling

Up until this point, I just threw some code on the table without going into detail. We've seen that RF are simply a bunch of DTs, trained on various datasets. Of course we had to choose differents subsets of our whole dataset, since training everything on the same data would be pointless.  
Let's look again at the sampling process:
```
def sampleDataset(X, y, nSamples):
    # Create a list of random indices
    indices = np.random.choice(len(X), nSamples)
    # Return the X and y samples
    return X[indices], y[indices]
```
This function just takes random samples from the dataset. It's not very sophisticated, but it works. Without explicitly telling it to, the function will take samples with replacement (default value of that parameter is **true**). This means that the same sample can be taken multiple times. This is not a problem, since we are not using the samples to train a single tree, but to train multiple trees. This is called **bootstrap sampling**. 
This is a very common way to sample data, and it is used in many other algorithms as well. As such, this is not a problem for us.  

### Training

Our training process assumes numerical values (which is true, in this case). We split the data until we run into our stopping criteria. We choose the classified class based on majority vote. These are concepts we've already discussed in the previous chapter, so now let's consider alternatives.  
We can use other stopping criteria, such as the number of samples in a node, or the number of nodes in a tree. "Number of samples in a node" means that when a node has less than a certain number of samples, we stop splitting. "Number of nodes in a tree" means that when a tree has more than a certain number of nodes, we stop splitting.  
We could think of other stopping criteria as well. For example, we could stop splitting when the entropy of a node is below a certain threshold, where entropy is defined as:
$$
H = -\sum_{i=1}^n p_i \log_2 p_i
$$
where $p_i$ is the probability of class $i$ in the node.  

As you can see, there are many ways to stop splitting.  

### Combining

We've seen that we can combine the predictions of the trees in the forest by taking the majority vote (the most common value is also called the *mode*). This is a very simple way to combine the predictions. In the case of continuous values, we can use any of the *average, mean or median*.  
We used the mode. Other options for categorical values are the *weighted mode* and the *weighted average*. These mean that we take the mode, but we weight the votes of each tree by the accuracy of that tree. This is a very simple way to improve the accuracy of the forest.  

One thing we should not miss while we're here is that increasing the number of trees is quite likely to increase our accuracy. A word of caution, though: always be wary of overfitting. DTs are quite prone to overfitting, which is why we prefer to have multiple trees, all independently trained on different subsets of the data. Imagine asking 1000 people their opinion on a certain topic. The more people you ask (considering they don't influence each other), the more likely you are to get a good answer. This is the same idea.


Now that we've considered some of the decision points that make a random forest work, let's see how sklearn performs.

In [30]:
# Import sklearn's random forest classifier
from sklearn.ensemble import RandomForestClassifier

In [31]:
# Use sklearn
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X, y)

RandomForestClassifier(max_depth=2, random_state=0)

In [32]:
# Print accuracy
print("Accuracy: ", clf.score(X, y))

Accuracy:  0.9666666666666667


We see that the accuracy is a lot higher (although it might be random, the difference should be around 0.2, so quite significant). Let's explore the default parameters to see how the library's solution differs from ours.  

Here are some of the more important parameters:
- n_estimators: the number of trees in the forest
- max_depth: the maximum depth of the tree
- criterion: the function to measure the quality of a split
- max_features: the number of features to consider when looking for the best split
- min_samples_split: the minimum number of samples required to split an internal node

Looking at default values, the major difference is the number of trees. We used 10, while sklearn uses 100. This is quite a big step-up. We set the max_depth to 2, and both criteria use "gini" (which is the default). The minimum number of samples to split is 2, which is the default. One difference then is that we we adopted the idea of "majority vote", while sklearn makes sure first that there is a minimum of 2 samples in each node. 




In [33]:
# RF with our code using 100 trees
Xsampled, ySampled = createRandomSamples(X, y, 100, 50)
trees = trainRandomForest(Xsampled, ySampled, 100, maxDepth = 2, majThreshold = 0.5)
predictions = predictRandomForest(trees, X)
print("Accuracy: ", np.sum(predictions == y) / len(y))

Accuracy:  0.6666666666666666


In our case, it seems that increasing trees doesn't work that well. Increasing majThreshold does instead increase our accuracy. This means our point of failure lies within the DT itself, namely in the way it's built. So let's explore that.

# Fine-tuning & Improving

Although we've left the Decision Tree chapter without giving much thought to improving performance, now is a good chance to explore weak points in our method and improve on them. In a real-world scenario, we would just use a framework's implementation (unless we're doing research, of course). However, we're still in the learning phase, so my goal is to work the **let's make it better** mindset. 



In an RF, decision trees are supposed to be as different as possible from each other. This is why we use different subsets of the data to train them. However, we're still using the same features to train them. This means that they are still quite similar. We can improve this by using a different set of features for each tree. This is called **feature bagging**. We'll create new functions that allow for this extra option. We'll then use them to train a new forest.

In [34]:
# New function to train DTs that takes a sample of features
def trainDecisionTree(X, y, maxDepth=1, majThreshold = 0.8, nFeatures = 2):
    # Create a new decision tree
    tree = DecisionTree(maxDepth, majThreshold)
    # Train it
    tree.fit(X, y, nFeatures)
    # Return it
    return tree
# New function to find best split of a feature, based on a sample of features
def find_best_split(X, y, features):
    best_gini = 1.0
    best_d, best_value = None, None
    for d in features: # for each feature
        value = np.mean(X[:, d]) # find the mean value
        X_left, X_right, y_left, y_right = split(X, y, d, value) # split the data based on the mean of that feature
        gini_left, gini_right = gini_impurity(y_left), gini_impurity(y_right) # calculate the gini impurity of the left and right splits
        gini = (len(y_left) / len(y)) * gini_left + (len(y_right) / len(y)) * gini_right # calculate the weighted gini impurity
        if gini < best_gini: # if the weighted gini impurity is less than the best gini impurity, update the best gini impurity (we have only 2 features, we could've calculated them directly but this is more general)
            best_gini = gini
            best_d = d # best feature index
            best_value = value # best value for that feature (the mean in our case)
    return best_d, best_value

In [35]:
# New DT class that uses max features parameter to train
class DecisionTree:
    def __init__(self, max_depth=2, minSamples = 2, maxFeatures = 1):
        self.max_depth = max_depth
        self.root = None
        self.minSamples = minSamples
        self.maxFeatures = maxFeatures
    def fit(self, X, y, maxFeatures):
        self.root = self._grow_tree(X, y)
    def _grow_tree(self, X, y, depth=0, maxFeatures = 1):
        num_samples_per_class = [np.sum(y == i) for i in range(y.max() + 1)]
        most_samples_per_class = np.max(num_samples_per_class)
        predicted_class = np.argmax(num_samples_per_class)
        node = Node(predicted_class=predicted_class)
        featureSampled = np.random.choice(X.shape[1], maxFeatures, replace=False)
        if depth < self.max_depth and most_samples_per_class > self.minSamples:
            d, value = find_best_split(X, y, featureSampled)
            if d is not None:
                X_left, X_right, y_left, y_right = split(X, y, d, value)
                node.d = d
                node.value = value
                node.left = self._grow_tree(X_left, y_left, depth + 1)
                node.right = self._grow_tree(X_right, y_right, depth + 1)
        return node
    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])
    def _traverse_tree(self, x, node):
        if node.left is None and node.right is None:
            return node.predicted_class
        if x[node.d] <= node.value:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)
    def accuracy(self, X, y):
        return np.sum(self.predict(X) == y) / len(y)

You can expand the code above to analyze it, if needed.

In [36]:
# Create a new RF class that uses max features parameter to train
class RandomForest:
    def __init__(self, n_trees=10, max_depth=2, minSamples = 2, maxFeatures = 1):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.minSamples = minSamples
        self.maxFeatures = maxFeatures
    def fit(self, X, y):
        self.trees = []
        for _i in range(self.n_trees):
            tree = DecisionTree(self.max_depth, self.minSamples, self.maxFeatures)
            tree.fit(X[_i], y[_i], self.maxFeatures)
            self.trees.append(tree)
    def predict(self, X):
        y_preds = np.empty([self.n_trees, len(X)])
        for i, tree in enumerate(self.trees):
            y_preds[i] = tree.predict(X)
        y_pred = []
        for sample_predictions in y_preds.T:
            y_pred.append(np.bincount(sample_predictions.astype('int')).argmax())
        return np.array(y_pred)
    def accuracy(self, X, y):
        return np.sum(self.predict(X) == y) / len(y)



In [56]:
# Train a RF with 100 trees, max depth 2, min samples 2 and max features 2
rf = RandomForest(n_trees=100, max_depth=2, minSamples = 2, maxFeatures = 2)
Xsampled, ySampled = createRandomSamples(X, y, 100, 30)
rf.fit(Xsampled, ySampled)


In [57]:
# Accuracy of rf
print("Accuracy: ", rf.accuracy(X, y))

Accuracy:  0.9266666666666666


There we go! Although not quite as good as the sklearn implementation, we've managed to improve our accuracy quite a bit. Remember, results vary (in my case, I went from 0.74 to around 0.9) due to the random nature of the sampling process.

### Remember

What I want you to remember from this chapter is that a Random Forest is only as good as it's trees. They have to be as different as possible from each other, so they come together to form one objective answer. Remember the 1000 people example. When we presented that idea earlier, we mentioned that those people should **not** influence each other. The way we would implement this within our own RF algorithms is by making sure that trees get different conditions to train on. Of course, we could go even further and look at other options to create more diversity in the trees. But I'll leave that option as an exercise for the reader.

Next chapter: [Ensemble Methods](ensembleMethods.ipynb)